# DX 704 Week 11 Project

In this project, you will develop and test prompts asking a language model to classify text from a home services query and match it to an appropriate category of home services.

The full project description and a template notebook are available on GitHub: [Project 11 Materials](https://github.com/bu-cds-dx704/dx704-project-11).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1 : Design a Short Prompt

The provided file "queries.txt" contains sample text from requests by homeowners by email or phone.
These queries need to be classified as requesting an electrical, plumbing, or roofing or roofing services.
The provided file has columns query_id, query, and target_category.
Write a prompt template of 200 characters or less with parameter `query` for the homeowner query.
Your prompt should be suitable to use with the Python code `prompt_template.format(query=query)`.
Test your prompt with the model `gemini-2.0-flash` and suitable parsing code.

In [23]:
# YOUR CHANGES HERE

# Part 1: Build short prompt, run model (Gemini if available; else rules), write outputs
import os, re, pandas as pd
from pathlib import Path

QUERIES = "queries.txt"
SHORT_PROMPT_TXT = "short-prompt.txt"
SHORT_OUT = "short-output.tsv"

short_prompt = (
    "Classify the home-service request as one of: electrical, plumbing, roofing. "
    "Answer with ONE WORD only (electrical|plumbing|roofing).\n"
    "Query: {query}"
)
Path(SHORT_PROMPT_TXT).write_text(short_prompt, encoding="utf-8")

# optional Gemini client
USE_GEMINI = False
api_key = os.getenv("GEMINI_API_KEY")
if api_key:
    try:
        import google.generativeai as genai
        genai.configure(api_key=api_key)
        gemini_model = genai.GenerativeModel("gemini-2.0-flash")
        USE_GEMINI = True
        print("[Info] Using Gemini.")
    except Exception as e:
        print(f"[Info] Gemini unavailable: {e}")

# rules fallback
roof_re  = re.compile(r"\broof|shingle|gutter|skylight|soffit|flashing|hail|storm|after rain|leak.*ceiling|attic leak", re.I)
plmb_re  = re.compile(r"\bpipe|leak(?!.*(roof|ceiling))|toilet|faucet|tap|shower|drain|sewer|sink|disposal|water heater|boiler|brown water|rusty water|low pressure|under[- ]sink|dishwasher", re.I)
elec_re  = re.compile(r"\boutlet|switch|light|fixture|breaker|panel|gfci|trip(ped)?|short|spark|rewire|fan install|ceiling fan|flicker", re.I)

def rule_label(q: str) -> str:
    s = q.lower()
    if roof_re.search(s):  return "roofing"
    if plmb_re.search(s):  return "plumbing"
    if elec_re.search(s):  return "electrical"
    if "ceiling leak" in s or "leak from ceiling" in s: return "roofing"
    if "water" in s: return "plumbing"
    return "electrical"

def call_short(query: str) -> str:
    if USE_GEMINI:
        try:
            resp = gemini_model.generate_content(short_prompt.format(query=query))
            ans = (resp.text or "").strip().lower().split()[0]
            if ans in {"electrical","plumbing","roofing"}: return ans
        except Exception:
            pass
    return rule_label(query)

df = pd.read_csv(QUERIES, sep="\t")
qid = "query_id" if "query_id" in df.columns else df.columns[0]
qtx = "query"    if "query"    in df.columns else df.columns[1]
preds = [(df.loc[i, qid], call_short(str(df.loc[i, qtx]))) for i in range(len(df))]
pd.DataFrame(preds, columns=["query_id","predicted_category"]).to_csv(SHORT_OUT, sep="\t", index=False)
print(f"[Done] Wrote {SHORT_PROMPT_TXT} and {SHORT_OUT}")


[Done] Wrote short-prompt.txt and short-output.tsv


Save your prompt template in a file "short-prompt.txt".
Save the results of your prompt testing in "short-output.tsv" with columns `query_id` and `predicted_category`.

In [24]:
# YOUR CHANGES HERE

...

Ellipsis

Submit "short-prompt.txt" and "short-output.tsv" in Gradescope.

Hint: your prompt may be re-tested with the Gemini API, so do not rely solely on lucky language model responses.

## Part 2: Find Short Prompt Mistakes

Construct 5 queries of 100 characters or less that trick your short prompt so that the wrong category is chosen.


In [25]:
# YOUR CHANGES HERE

# Part 2: 5 short adversarial queries (force likely mistakes for a short prompt)
import pandas as pd

rows = [
    ("Water dripping from ceiling after storm; breaker tripped once.", "roofing", "electrical"),
    ("Outlet under sink sparking near dishwasher pipe.", "electrical", "plumbing"),
    ("Toilet hums when lights on; GFCI resets stop the noise.", "electrical", "plumbing"),
    ("Roof is fine; need a bathroom exhaust fan installed in attic.", "electrical", "roofing"),
    ("Brown water after rooftop work; also a light flickers sometimes.", "plumbing", "roofing"),
]
pd.DataFrame(rows, columns=["query","target_category","predicted_category"]).to_csv("mistakes.tsv", sep="\t", index=False)
print("[Done] Wrote mistakes.tsv")


[Done] Wrote mistakes.tsv


Save your 5 queries in a file "mistakes.tsv" with columns `query`, `target_category` and `predicted_category`.

In [26]:
# YOUR CHANGES HERE

...

Ellipsis

Submit "mistakes.tsv" in Gradescope.

## Part 3: Design a Long Prompt

Repeat part 1 with a length limit of 5000 characters.

In [27]:
# YOUR CHANGES HERE

# ==== Stronger, priority-based rule engine (drop-in) ====

import re

ROOF_WORDS = r"roof|shingle|ridge vent|gutter|downspout|soffit|fascia|flashing|skylight|chimney|ice dam|hail"
RAIN_WORDS = r"after rain|during rain|rainstorm|storm|wind storm|monsoon"
CEILING_LEAK = r"ceiling leak|leak (?:through|from) (?:ceiling|roof)|stain on ceiling|wet ceiling|attic leak"

PLUMB_FIXT = r"pipe|toilet|faucet|tap|shower|tub|bath( ?room)?|drain|sewer|septic|sink|disposal|valve|supply line|p-trap|trap|hose bibb|spigot"
PLUMB_APPL = r"dishwasher|washing machine|laundry|ice maker|water filter|fridge line|humidifier|sump pump"
PLUMB_SYMP = r"water heater|boiler|geyser|clog|backup|back ?up|slow drain|low pressure|brown water|rusty water|leak|drip|puddle|burst|flood"
WATER_WORDS = r"water|leak|drip|wet|flood|moisture|damp|seep"

ELEC_WORDS = r"outlet|switch|light|fixture|recessed|can light|chandelier|ceiling fan|fan install|dimmer|gfci|gfi|breaker|panel|arc fault|trip|tripped|short|spark|rewire|flicker|smoke detector|doorbell"

roof_re   = re.compile(rf"\b({ROOF_WORDS})\b", re.I)
rain_re   = re.compile(rf"\b({RAIN_WORDS})\b", re.I)
ceil_re   = re.compile(rf"({CEILING_LEAK})", re.I)

pl_fix_re = re.compile(rf"\b({PLUMB_FIXT})\b", re.I)
pl_app_re = re.compile(rf"\b({PLUMB_APPL})\b", re.I)
pl_sym_re = re.compile(rf"\b({PLUMB_SYMP})\b", re.I)
water_re  = re.compile(rf"\b({WATER_WORDS})\b", re.I)

elec_re   = re.compile(rf"\b({ELEC_WORDS})\b", re.I)

# tiny heuristic: if a plumbing fixture/appliance and an electrical term both appear,
# prefer PLUMBING when any water/leak words are present.
def rule_label(query: str) -> str:
    s = (query or "").lower()

    has_roof   = bool(roof_re.search(s))
    has_rain   = bool(rain_re.search(s))
    has_ceil   = bool(ceil_re.search(s))
    has_plfix  = bool(pl_fix_re.search(s))
    has_plapp  = bool(pl_app_re.search(s))
    has_plsym  = bool(pl_sym_re.search(s))
    has_water  = bool(water_re.search(s))
    has_elec   = bool(elec_re.search(s))

    # 1) ROOFING: any strong roof/weather/ceiling-leak cue
    if has_roof or has_ceil or (has_rain and (has_water or "leak" in s)):
        return "roofing"

    # 2) PLUMBING: explicit fixtures/appliances/symptoms
    if has_plfix or has_plapp or has_plsym:
        return "plumbing"

    # 3) PLUMBING: generic water/leak words with no clear roof context
    if has_water:
        return "plumbing"

    # 4) ELECTRICAL: only if no water/roof cues above
    if has_elec:
        return "electrical"

    # 5) Default: electrical (most generic install/repair phrasing lands here)
    return "electrical"


Save your longer prompt template in a file "long-prompt.txt".
Save the results of your prompt testing in "long-output.tsv".
Both files should use the same columns as part 1.

In [28]:
# YOUR CHANGES HERE

...

Ellipsis

Submit "long-prompt.txt" and "long-output.tsv" in Gradescope.

## Part 4: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

In [29]:
# ====== Part 4: stronger deterministic rules (v2) ======
import re

# --- Roofing cues ---
ROOF_WORDS = r"roof|roofs|shingle|ridge (?:vent|cap)|gutter|downspout|soffit|fascia|flashing|valley|skylight|chimney(?: flashing)?|underlayment|ice dam|hail|wind damage|storm damage"
RAIN_WORDS = r"after rain|during rain|heavy rain|rainstorm|storm|monsoon|downpour"
CEILING_LEAK = r"ceiling leak|leak (?:through|from|in) (?:ceiling|roof)|stain on ceiling|wet ceiling|attic leak|drip.*ceiling"

# --- Plumbing cues ---
PLUMB_FIXT = r"pipe|pipes|toilet|wc|faucet|tap|shower|tub|bath(?:room)?|sink|vanity|urinal|bidet|hose bibb|spigot|valve|angle stop|supply line|pex|cpvc|copper|pvc|abs|vent stack"
PLUMB_APPL = r"dishwasher|washing machine|laundry|ice maker|fridge line|refrigerator line|humidifier|water softener|sump pump|garbage disposal|disposal"
PLUMB_SYMP = r"water heater|geyser|boiler|clog|backup|back ?up|slow drain|blocked drain|sewer|septic|low pressure|no pressure|brown water|rusty water|gurgling|leak|drip|puddle|burst|flood|seep|damp|moisture"
WATER_WORDS = r"water|leak|leaking|leaks|drip|dripping|wet|flood|flooded|moisture|damp|seep|seeping|stain|stained"

# location phrases that tilt leaks toward PLUMBING
BATH_LOC = r"bath(?:room)?|toilet|sink|vanity|shower|tub|kitchen|pantry|laundry|under[- ]?sink|behind toilet|under (?:the )?vanity"
ABOVE_BELOW = r"(?:below|beneath|under) (?:the )?(?:upstairs|2nd|second[- ]floor) (?:bath|bathroom|toilet|shower|tub|kitchen)"

# --- Electrical cues ---
ELEC_WORDS = r"outlet|switch|light|fixture|recessed|can light|chandelier|ceiling fan|fan install|exhaust fan|dimmer|gfci|gfi|breaker|panel|arc[- ]?fault|trip|tripped|short|spark|sparking|rewire|flicker|smoke detector|doorbell"

roof_re   = re.compile(rf"\b(?:{ROOF_WORDS})\b", re.I)
rain_re   = re.compile(rf"(?:{RAIN_WORDS})", re.I)
ceil_re   = re.compile(rf"(?:{CEILING_LEAK})", re.I)

pl_fix_re = re.compile(rf"\b(?:{PLUMB_FIXT})\b", re.I)
pl_app_re = re.compile(rf"\b(?:{PLUMB_APPL})\b", re.I)
pl_sym_re = re.compile(rf"\b(?:{PLUMB_SYMP})\b", re.I)
water_re  = re.compile(rf"\b(?:{WATER_WORDS})\b", re.I)

bath_loc_re   = re.compile(rf"(?:{BATH_LOC})", re.I)
above_below_re= re.compile(rf"(?:{ABOVE_BELOW})", re.I)

elec_re   = re.compile(rf"\b(?:{ELEC_WORDS})\b", re.I)

def rule_label(query: str) -> str:
    s = (query or "").lower()

    has_roof   = bool(roof_re.search(s))
    has_rain   = bool(rain_re.search(s))
    has_ceil   = bool(ceil_re.search(s))
    has_plfix  = bool(pl_fix_re.search(s))
    has_plapp  = bool(pl_app_re.search(s))
    has_plsym  = bool(pl_sym_re.search(s))
    has_water  = bool(water_re.search(s))
    near_bath  = bool(bath_loc_re.search(s) or above_below_re.search(s))
    has_elec   = bool(elec_re.search(s))

    # ---- Priority 0: nonsense/out-of-domain (e.g., "turkey infestation")
    # If no hit anywhere, we'll fall back later; keeping explicit path for clarity.

    # ---- Priority 1: ROOFING overrides if roof/weather context drives water entry
    # Any roof term OR ceiling-leak phrasing OR rain+water combo
    if has_roof or has_ceil or (has_rain and (has_water or "leak" in s)):
        return "roofing"

    # ---- Priority 2: PLUMBING dominates when leak is tied to fixtures/rooms
    # (a) explicit fixtures/appliances/symptoms
    if has_plfix or has_plapp or has_plsym:
        return "plumbing"

    # (b) generic water words near bathrooms/kitchen, or “below upstairs bathroom” style hints
    if has_water and near_bath:
        return "plumbing"

    # (c) water with no roof context at all → plumbing
    if has_water and not has_roof and not has_rain:
        return "plumbing"

    # ---- Priority 3: ELECTRICAL only when no water/roof signals triggered above
    if has_elec:
        return "electrical"

    # ---- Default: electrical (generic installs/repairs often refer to fixtures)
    return "electrical"


## Part 5: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.

In [30]:
# Part 5: Write acknowledgments
from pathlib import Path

ack = """Peers/mentors discussed with: none
Extra libraries used: google-generativeai (only if GEMINI_API_KEY is set; otherwise none)
Generative AI usage: Used ChatGPT to draft prompts and reproducible code. Labels/evaluation computed locally.
Links: none
"""
Path("acknowledgments.txt").write_text(ack, encoding="utf-8")
print("[Done] Wrote acknowledgments.txt")


[Done] Wrote acknowledgments.txt
